# Parse the PHREEC databases

In [3]:
def database_parsing(db):
    print(f'\n\n\n{db}')
    
    database = pandas.read_table(db, sep='\n')
    start_master = False
    if database.columns == ['SOLUTION_MASTER_SPECIES']:
        start_master = True
    database.columns = ['content']

    elements_rows = []
    minerals_rows = []
    elemental_parsing = False
    mineral_parsing = False
    for index, row in database.iterrows():
        if (re.search('SOLUTION_MASTER_SPECIES', row['content']) or start_master) and not elemental_parsing :
            while not re.search('SOLUTION_SPECIES', database.at[index, 'content']):
                split_row = database.at[index, 'content'].split()
                if all(not re.search('^#', entity) for entity in split_row):
                    elements_rows.append(split_row)
                index+= 1
            elemental_parsing = True

        if re.search('PHASES', row['content']) and not mineral_parsing:
            loop = False
            while not re.search('PITZER|EXCHANGE_MASTER_SPECIES|SURFACE_MASTER_SPECIES', database.at[index, 'content']):
                if not loop:
                    minerals_rows.append(['phases', 'formula'])
                    loop = True
                    index += 1
                    continue

                if re.search('(^\w+\s*\d*$)',database.at[index, 'content']):
                    reactants = database.at[index+1, 'content'].split(' = ')[0]
                    if all('#' not in entity for entity in reactants):
                        formula = reactants.split('+')[0].strip()
                        name = database.at[index, 'content']
                        name = re.sub('(\s+\d*)', '', name)
                        minerals_rows.append([name, formula])
                index+= 1
                
                if index == len(database):
                    break
            mineral_parsing = True

    # define the elements content for the database
    elements = pandas.DataFrame(elements_rows)
    elements.fillna(' ')
#     elements.columns = elements.iloc[0]
    elements.drop([0], inplace = True)
    for column in elements:
        nan_entries = 0
        alphanumeric_entries = 0
        for entry in elements[column]:
#             if entry in [None, ' ', nan]:
            if entry is not None:
                if re.search('[a-z]|[0-9]', entry, re.IGNORECASE) and entry is not None:
    #             if entry is str or entry is float or entry is int:
                    alphanumeric_entries += 1
                else:
                    nan_entries += 1
            else:
                nan_entries += 1
        if nan_entries > alphanumeric_entries and len(elements.columns) > 5:
            print('deleted column: ', column)
            del elements[column]
    
    print(elements)
    elements.columns = ['elements', 'species', 'alk', 'gfw_formula', 'element_gfw']
#     elements.rename(columns = {'SOLUTION_MASTER_SPECIES':'elements'}, inplace = True)
    print(elements)
#         elements = elements.iloc[pandas.RangeIndex(len(elements)).drop([x for x in range(4)])]
    elements_list = list(elements['elements'])

    # define the minerals content for the database
    minerals = pandas.DataFrame(minerals_rows)
    minerals.columns = minerals.iloc[0]
    minerals = minerals.drop(0)
    print(minerals)
    mineral_list = list(minerals['phases'])
    formula_list = list(minerals['formula'])
    
    return elements, minerals

# Calculate the compound masses 

In [ ]:
from chemicals import periodic_table
import pandas
import json
import re


# define elemental masses
elemental_masses = {}
for element in periodic_table:
    elemental_masses[element.symbol] = element.MW

class CpdMassPkg():
    def __init__(self):
        self.group_masses = {}
        pass

    def mineral_mass(self, mineral_formula, mineral_name = None):
        self.formula = mineral_formula
        skip_characters = self.total_mass = mass = self.group_number = self.sub_group_number = 0
        stoich = ''
        for ch_number in range(len(self.formula)):
            print('total_mass', self.total_mass)
            print('ch_number', ch_number)
            print('character', self.formula[ch_number])    
            
            self.parse_character(ch_number)
                
        print(f'{self.formula} mass: {self.total_mass}')
        self.clear()
        return self.total_mass

    def parse_character(self, ch_number, group = False):
        self.group_masses = {}
        if re.search('[A-Z(): ]',self.formula[ch_number]):
            if self.element:
                if self.stoich = '':
                    self.stoich = 1
                elif float(self.stoich):
                    if self.group_masses['h2o']:
                        self.group_masses['h2o']['mass'] += element_masses[self.element] * float(self.stoich)
                    else:
                        self.total_mass += element_masses[self.element] * float(self.stoich)
                else:
                    print(f'--> ERROR: The stoich {self.stoich} is unexpected.')
    
            if re.search('[(): ]', self.formula[ch_number]):
                self.element = None
                if self.stoich == '':
                    self.stoich = 1
                    if self.group_masses != {}:
                        self.group_masses[self.group_number][self.sub_group_number]['mass'] += mass
                    self.total_mass += element_masses[self.element] * float(self.stoich)
                if self.formula[ch_number] == '(':
                    if self.group_masses[self.group_number][self.sub_group_number]['done']:
                        sub_group_mass = self.group_masses[self.group_number][self.sub_group_number]['mass'] * self.group_masses[self.group_number][self.sub_group_number]['stoich']
                        self.group_masses[self.group_number][self.sub_group_number] += sub_group_mass
                        if self.sub_group_number - 1 in self.group_masses[self.group_number]:
                            self.sub_group_number -= 1
                    
                    if self.group_masses[self.group_number][self.sub_group_number]:
                        self.sub_group_number += 1
                        self.group_masses[self.group_number][self.sub_group_number] = {'stoich': '', 'mass': '', 'done':False}
                    elif self.group_masses[self.group_number]:
                        self.group_number += 1
                        self.sub_group_number = 0
                        self.group_masses[self.group_number][self.sub_group_number] = {'stoich': '', 'mass': '', 'done':False}
                    else:
                        self.group_masses[self.group_number][self.sub_group_number] = {'stoich': '', 'mass': '', 'done':False}
                    
                elif self.formula[ch_number] == ':':
                    print('element', formula[ch_number])
                    self.group_masses['h2o'] = {'stoich': '', 'mass': ''}
                elif self.formula[ch_number] == ' ':
                    pass
                elif self.formula[ch_number] == ')':
                    self.group_masses[self.group_number][self.sub_group_number]['done'] = True
                    self.sub_group_number -= 1

            self.element = self.formula[ch_number]
            self.stoich = ''
        elif re.search('[a-z]',self.formula[ch_number]):
            self.element += formula[ch_number]
        elif re.search('[0-9\.]', self.formula[ch_number]):
            if self.group_masses['h2o']:
                self.group_masses['h2o']['stoich'] += self.formula[ch_number]
            else:
                self.stoich += self.formula[ch_number]
        else:
            print(f'--> ERROR: The {self.formula} formula is not predictable.')
    
        final_mass2 = group_mass2 * stoich
        ch_no2 += 1
        
    def clear(self,):
        self.group_masses = {}
        self.element = self.stoich = self.formula = self.total_mass = None

In [37]:
from glob import glob
# %run compound_masses.py

cpd_mass = CpdMassPkg()

for database in glob('../ROSS/rosspy/databases/*.dat'):
    database_json = {'elements': {}, 'minerals': {}}
    elements, minerals = database_parsing(database)
    
    # create the elements JSON
    for index, element in elements.iterrows():
        database_json['elements'][element['elements']] = {'charge_specie': element['alk'], 'gfw_formula':element['gfw_formula'], 'element_gfw':element['element_gfw']}
    
    # create the mienrals JSON
    for index, mineral in minerals.iterrows():
        print('\n\n')
        print(mineral['formula'], '\n','='*len(mineral['formula']))
        mineral['formula'] = re.sub('Cyanide|Cyanate', 'CN', mineral['formula'])
        if re.search('PHASES', mineral['phases']):
            continue
        database_json['minerals'][mineral['phases']] = {
            'formula': mineral['formula'], 
            'mass': cpd_mass.mineral_mass(mineral['formula'], mineral['phases'])
        }
        
    # export the JSON files
    database_json_name = re.sub('.dat$', '.json', database)
    with open(database_json_name, 'w') as output:
        json.dump(database_json, output, indent = 4)




../ROSS/rosspy/databases\Amm.dat
             0       1     2              3        4
1            H      H+  -1.0              H    1.008
2         H(0)      H2     0              H     None
3         H(1)      H+  -1.0              0     None
4            E      e-     0            0.0        0
5            O     H2O     0              O     16.0
6         O(0)      O2     0              O     None
7        O(-2)     H2O     0              0     None
8           Ca    Ca+2     0             Ca    40.08
9           Mg    Mg+2     0             Mg   24.312
10          Na     Na+     0             Na  22.9898
11           K      K+     0              K   39.102
12          Fe    Fe+2     0             Fe   55.847
13      Fe(+2)    Fe+2     0             Fe     None
14      Fe(+3)    Fe+3  -2.0             Fe     None
15          Mn    Mn+2     0             Mn   54.938
16      Mn(+2)    Mn+2     0             Mn     None
17      Mn(+3)    Mn+3     0             Mn     None
18        

TypeError: cannot unpack non-iterable NoneType object

In [ ]:
from chemicals import periodic_table
import pandas
import json
import re


# define elemental masses
elemental_masses = {}
for element in periodic_table:
    elemental_masses[element.symbol] = element.MW

class CpdMassPkg():
    def __init__(self):
        self.group_masses = {}
        pass

    def mineral_mass(self, mineral_formula, mineral_name = None):
        self.formula = mineral_formula
        skip_characters = total_mass = mass = self.group_mass_number = 0
        stoich = ''
        for ch_number in range(len(self.formula)):
            self.group_masses[mineral] = {}
            if re.search('[A-Z]',formula[ch_number]):
                if element:
                    if stoich = '':
                        stoich = 1
                    else:
                        print(f'--> ERROR: The stoich {stoich} is unexpected.')
                    total_mass += element_masses[element] * float(stoich)
                if 
                    
                element = formula[ch_number]
            elif re.search('[a-z]',formula[ch_number]):
                element += formula[ch_number]
            elif re.search('[0-9\.]', formula[ch_number]):
                if self.group_masses[mineral]['h2o']:
                    self.group_masses[mineral]['h2o']['stoich'] += formula[ch_number]
                else:
                    stoich += formula[ch_number]
            elif re.search('[(: ]', formula[ch_number]):
                element = None
                if stoich == '':
                    stoich = 1
                    total_mass += element_masses[element] * float(stoich)
                if formula[ch_number] == '(':
                    self.group_mass_number += 1
                    self.group_parsing(self.formula, ch_number)
                elif formula[ch_number] == ':':
                    print('element', formula[ch_number])
                    self.group_masses[mineral]['h2o'] = {'stoich': '', 'mass': ''}
                elif formula[ch_number] == ' ':
#                     total_skips += 1
                    continue
            else:
                print(f'--> ERROR: The {formula} formula is not predictable.')

            print('total_mass', total_mass)
            print('ch_number', ch_number)
            print('character', self.formula[ch_number])
            if self.formula[ch_number] == '(':
                
            else:
                print('element', self.formula[ch_number])
                skip_characters, mass = self.parse_formula(self.formula, ch_number, final)
                total_mass += mass

        print(f'{self.formula} mass: {total_mass}')
        return total_mass

    def group_parsing(self, formula, ch_number, final):
        final_mass2 = total_skips = total_loops = skips2 = 0
        ch_no2 = ch_number + 1
        group = True
        while formula[ch_no2] != ')' and ch_no2 != len(formula)-1:
            total_loops += 1
            print('ch_number', ch_no2)
            print('character', formula[ch_no2])
            if re.search('[\(\s]', formula[ch_no2]):
                ch_no2 += 1
                total_skips += 1
                if formula[ch_no2] == '('
                    self.group_mass_number += 1
                print('total_skips1', total_skips)
                continue

            if self.group_mass_number > 1:
                skips, mass = self.parse_formula(formula, ch_no2, final, group = group)
                self.group_masses[mineral][self.group_mass_number] += mass
                ch_no2 += 1
                total_skips += 1
                if formula[ch_no2] == ')':
                    print('group_mass2', group_mass2)
                    skips, stoich = self.parse_stoich(formula, ch_no2 + 1)
                    print('group_stoich2', stoich)
                    final_mass2 = group_mass2 * stoich
                    ch_no2 += 1
                total_skips += skips
                self.group_mass_number -= 1
                print('total_skips2', total_skips)

            else:
                print(formula[ch_no2])
                skips, mass = self.parse_formula(formula, ch_no2, final, group = group)
                self.group_masses[mineral][self.group_mass_number] += mass
                ch_no2 += 1
                total_skips += skips
                print('total_skips3', total_skips)

        print('total_loops', total_loops)

# Brainstorming

In [3]:
import rosspy
ross = rosspy.ROSSPkg()

for dic in ['parameters', 'variables']:
    print(type(ross.dic))

AttributeError: 'ROSSPkg' object has no attribute 'dic'

In [1]:
if type(12) in [int, float]:
    print('yes')

yes
